<a href="https://colab.research.google.com/github/jogg1989/MCAPTI1Tarea2/blob/main/Tarea2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica 2

**Nombre:** Jorge Osvaldo Gonzalez Gonzalez  
**e-mail:** jorge.gonzalez@alumnos.udg.mx

### MODULES

In [46]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

### CLASES

In [47]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

### FUNCTIONS

In [48]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW)

* Cargar trayectorias en **Pandas** Data Frame
* Calcular métrica utilizando exclusivamente funciones de NumPy
* Guardar métricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [49]:
# Load existing trajectories
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv') 

In [57]:
# Compute path length
## start - Add your code here
dis_bm2_s3 = pd.DataFrame([{"dis": 0}])

for i in range(1, BM_2d_df_3.shape[0]):
    pointA = np.array((BM_2d_df_3.iloc[i-1].x_pos, BM_2d_df_3.iloc[i-1].y_pos)) 
    pointB = np.array((BM_2d_df_3.iloc[i].x_pos, BM_2d_df_3.iloc[i].y_pos))
    dis = round(np.sqrt(np.sum(np.square(pointA - pointB))), 3)
    dis_bm2_s3 = pd.concat([dis_bm2_s3, pd.DataFrame([{"dis": dis_bm2_s3.iloc[i-1].dis + dis}])], ignore_index=True)

## end - Add your code here


        dis
0       0.0
1       3.0
2       6.0
3       9.0
4      12.0
..      ...
995  2985.0
996  2988.0
997  2991.0
998  2994.0
999  2997.0

[1000 rows x 1 columns]


In [60]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
## start - Add your code here
fig_path_length.add_trace(
        go.Scatter(
            x = np.arange(len(dis_bm2_s3)),
            y = dis_bm2_s3.dis,
            line = dict(width=2),
            mode = 'lines'
        )
    )
## end - Add your code here


# Second trace BM2
## start - Add your code here
    
## end - Add your code here


# Third trace CRW
## start - Add your code here
    
## end - Add your code here


fig_path_length.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW)

* Cargar trayectorias en **Pandas** Data Frame
* Guardar metricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [ ]:
# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [ ]:
# Empty MSD_BM
MSD_BM = np.empty(shape=(0))

# MSD for BM_2d_df_6
for tau in range(1,BM_2d_df_6.shape[0]):
    ## start - Add your code here
    
    ## end - Add your code here

# Empty MSD_CRW
MSD_CRW = np.empty(shape=(0))
# MSD for CRW_2d_df_9
for tau in range(1,CRW_2d_df_9.shape[0]):
    ## start - Add your code here
    
    ## end - Add your code here
    
# Save metrics to Dataframe
## start - Add your code here
    
## end - Add your code here

# Write to csv
## start - Add your code here
    
## end - Add your code here

In [ ]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
## start - Add your code here
    
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
    
## end - Add your code here


fig_path_length.show()